In [1]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
import backtrader as bt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

In [2]:
#设置数据文件路径
datapath0=r'/Users/nilei/Data/12M8-202007/EURUSD_D1.csv'
datapath1=r'/Users/nilei/Data/12M8-202007/EURUSD_H4.csv'


In [4]:
from dateutil import parser
def getDateTime(s):
    d = parser.parse(s)
    return d
df0=pd.read_csv(datapath0)   #datapath0
df0.index=df0['Time (UTC)'].apply(lambda x:getDateTime(x))
old=df0.columns[1:]
new=['open','high','low','close','volume']
df0=df0.rename(columns=dict(zip(old,new)))
df0=df0.iloc[:,1:]
df0['openinterest']=0
df0.head()

,open,high,low,close,volume,openinterest
Time (UTC),,,,,,
2010-01-01,1.43283,1.43356,1.43181,1.43335,76787.4,0
2010-01-03,1.43295,1.43359,1.42910,1.43141,6331.3,0
2010-01-04,1.43143,1.44556,1.42559,1.44244,80019.4,0
2010-01-05,1.44238,1.44834,1.43445,1.43634,79887.1,0
2010-01-06,1.43638,1.44342,1.42807,1.44005,80971.8,0


In [6]:
from dateutil import parser
def getDateTime(s):
    d = parser.parse(s)
    return d
df1=pd.read_csv(datapath1)   #datapath1
df1.index=df1['Time (UTC)'].apply(lambda x:getDateTime(x))
old=df1.columns[1:]
new=['open','high','low','close','volume']
df1=df1.rename(columns=dict(zip(old,new)))
df1=df1.iloc[:,1:]
df1['openinterest']=0
df1.head()

,open,high,low,close,volume,openinterest
Time (UTC),,,,,,
2010-01-01 00:00:00,1.43283,1.43306,1.43181,1.43209,9999.4,0
2010-01-01 04:00:00,1.43218,1.43346,1.43182,1.43324,12324.1,0
2010-01-01 08:00:00,1.43297,1.43346,1.43218,1.43306,18182.8,0
2010-01-01 12:00:00,1.43321,1.43354,1.43207,1.43337,15405.0,0
2010-01-01 16:00:00,1.43337,1.43356,1.43225,1.43342,13920.0,0


In [16]:
#设置回测时间段
import datetime as datetime
data0 = bt.feeds.PandasData(dataname=df0,                               
                            fromdate=datetime.datetime(2019, 1, 1),                               
                            todate=datetime.datetime(2020, 1, 1) ) 
data1 = bt.feeds.PandasData(dataname=df1,                               
                            fromdate=datetime.datetime(2019, 12, 1),                               
                            todate=datetime.datetime(2020, 1, 1) ) 

In [ ]:
class MyStrategy(bt.Strategy):
    params = (
        ('ssa_window', 15),
        ('maperiod', 15),
    )
 
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
 
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
 
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)
 
 
    def start(self):
        print("the world call me!")
 
    def prenext(self):
        print("not mature")
 
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
 
        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
 
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
 
            self.bar_executed = len(self)
 
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
 
        self.order = None
 
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
 
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
 
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
 
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
 
        # Check if we are in the market
        if not self.position:
 
            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:
 
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
 
        else:
 
            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
    def stop(self):
        print("death")
 
 

In [14]:
class MyStrategy(bt.Strategy):
    params = dict(multi=True)

    def __init__(self):
        self.pp = pp = bt.ind.PivotPoint(self.data1)
        pp.plotinfo.plot = False  # deactivate plotting

        if self.p.multi:
            pp1 = pp()  # couple the entire indicators
            self.sellsignal = self.data0.close < pp1.s1
        else:
            self.sellsignal = self.data0.close < pp.s1()

    def next(self):
        txt = ','.join(
            ['%04d' % len(self),
             '%04d' % len(self.data0),
             '%04d' % len(self.data1),
             self.data.datetime.date(0).isoformat(),
             '%.2f' % self.data0.close[0],
             '%.2f' % self.pp.s1[0],
             '%.2f' % self.sellsignal[0]])

        print(txt)

In [ ]:
def runstrat():
    args = parse_args()

    cerebro = bt.Cerebro()
    data = btfeeds.BacktraderCSVData(dataname=args.data)
    cerebro.adddata(data)
    cerebro.resampledata(data, timeframe=bt.TimeFrame.Months)

    cerebro.addstrategy(St, multi=args.multi)

    cerebro.run(stdstats=False, runonce=False)
    if args.plot:
        cerebro.plot(style='bar')


def parse_args():
    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
        description='Sample for pivot point and cross plotting')

    parser.add_argument('--data', required=False,
                        default='../../datas/2005-2006-day-001.txt',
                        help='Data to be read in')

    parser.add_argument('--multi', required=False, action='store_true',
                        help='Couple all lines of the indicator')

    parser.add_argument('--plot', required=False, action='store_true',
                        help=('Plot the result'))

    return parser.parse_args()


if __name__ == '__main__':
    runstrat()

关键点：resampledata

In [21]:
def main():
    #关键点
    cerebro = bt.Cerebro(stdstats=False)      
    cerebro.addstrategy(MyStrategy)    
    #将数据加载至回测系统
    cerebro.adddata(data1)
    data2=cerebro.resampledata(data1, timeframe=bt.TimeFrame.Days)
    cerebro.run()
    #cerebro.plot(style='bar')

In [22]:
main()

IndexError: array index out of range

In [ ]:
if __name__ == '__main__':
 
    # Create a cerebro entity
    cerebro = bt.Cerebro(stdstats=False)
 
 
    # Add a strategy
    cerebro.addstrategy(MyStrategy)
 
    dataframe = pd.read_csv('RB.SHF.csv', index_col=0, parse_dates=True)
    data0 = bt.feeds.PandasData(dataname=dataframe,
                                fromdate=datetime.datetime(2014, 5, 13),
                                todate=datetime.datetime(2014, 6, 20),
                                timeframe=bt.TimeFrame.Minutes
                                )
    # Add the Data Feed to Cerebro
    cerebro.adddata(data0)
 
    data2 = cerebro.resampledata(data0, timeframe=bt.TimeFrame.Days)
    cerebro.run()
    cerebro.plot(style='bar')